In [59]:
import sys
sys.path.append("../")

from SALib.sample import saltelli
from network_model import *
from tqdm import tqdm
import numpy as np
import pickle
import lzma
import os


import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({"font.size" : 15,
                     "figure.dpi" : 100, 
                     "grid.alpha" : 0.3, 
                     "axes.grid": True, 
                     "axes.axisbelow" : True,
                     "figure.figsize":(8,6),
                     "mathtext.fontset":"cm",
                     "xtick.labelsize": 14,
                     "ytick.labelsize": 14,
                     "axes.labelsize": 16, 
                     "legend.fontsize": 13.5})
USE_TEX = False
if USE_TEX:
    plt.rc("text", usetex=True)
    plt.rc("text.latex", preamble=r"""
     \usepackage{times}
     \usepackage{mathptmx}""")
else:
    plt.rc("text", usetex=False)
plt.rc("font", family="serif")

In [60]:
def category_1(data):
    decreasing = []
    for i in range(len(data)):
        d = data[i]
        mono_dec = True
        for row in d:
            if max(row[1:] - row[:-1]) > 0:
                mono_dec = False
                break
        if mono_dec:
            decreasing.append(i)
    return decreasing


def category_2(data, ignore):
    increase_less_initial = []
    for i in range(len(data)):
        if i not in ignore:
            d = data[i]
            if np.all(np.max(data[i], axis=1) == data[i][:,0]):
                increase_less_initial.append(i)
    return increase_less_initial


def category_3(data, ignore):
    surpass_end_poor = []
    for i in range(len(data)):
        if i not in ignore:
            if max(data[i].T[-1]) < 0.1:
                surpass_end_poor.append(i)
    return surpass_end_poor


def category_4(data, ignore):
    two_equilibria_less_initial = []
    for i in range(len(data)):
        if i not in ignore:
            if np.all(data[i].T[-1] < data[i].T[0]):
                two_equilibria_less_initial.append(i)
    return two_equilibria_less_initial


def category_5(data, ignore):
    two_equilibria_richer = []
    for i in range(len(data)):
        if i not in ignore:
            two_equilibria_richer.append(i)
    return two_equilibria_richer

In [61]:
directories = ["./concat_W_arrays", 
               "./concat_W_arrays_random", 
               "./concat_W_arrays_cpt",
               "./concat_W_arrays_cpt_random",
               "./concat_W_arrays_risk_aversion_experiment/",
               "./concat_W_arrays_sda",
               "./concat_W_arrays_cpt_sda"]

labels = ["MPT (Holme-Kim)", 
          "MPT (Random)", 
          "CPT (Holme-Kim)",
          "CPT (Random)",
          "MPT (Holme-Kim) + Low RA",
          "MPT (SDA)",
          "CPT (SDA)"]

communities_file = ["./augmented_communities.pickle", 
                    "./augmented_communities_random.pickle",
                    "./augmented_communities.pickle",
                    "./augmented_communities_random.pickle",
                    "./augmented_communities.pickle"]

# Analysis at agent level

In [62]:
RESULTS = {}

for dir_idx, D in enumerate(directories):

    print(f"Running analysis for {labels[dir_idx]}...")

    category_counts = {i:[] for i in range(5)}

    for f in tqdm(os.listdir(D)):

        data = pickle.load(lzma.open(os.path.join(D,f)))

        cat1 = category_1(data)
        cat2 = category_2(data, cat1)
        cat3 = category_3(data, cat1+cat2)
        cat4 = category_4(data, cat1+cat2+cat3)
        cat5 = category_5(data, cat1+cat2+cat3+cat4)
        assert len(cat1+cat2+cat3+cat4+cat5) == len(data)

        for i,cat in enumerate([cat1, cat2, cat3, cat4, cat5]):
            category_counts[i].append(len(cat))

    RESULTS[labels[dir_idx]] = category_counts
    print("--- RESULTS ---")
    print(category_counts)
    print()

Running analysis for MPT (Holme-Kim)...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [07:18<00:00, 43.84s/it]


--- RESULTS ---
{0: [3595, 3907, 3942, 4098, 2836, 4255, 4554, 4017, 3712, 3929], 1: [475, 624, 1111, 84, 553, 370, 764, 1311, 471, 543], 2: [4972, 4391, 3924, 4879, 5541, 4432, 3741, 3528, 4652, 4536], 3: [42, 45, 41, 40, 74, 26, 26, 48, 29, 29], 4: [132, 249, 198, 115, 212, 133, 131, 312, 352, 179]}

Running analysis for MPT (Random)...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [07:20<00:00, 44.07s/it]


--- RESULTS ---
{0: [3595, 2836, 3712, 4017, 3907, 4255, 3942, 4098, 3929, 4554], 1: [475, 553, 471, 1311, 624, 370, 1111, 84, 543, 764], 2: [4972, 5541, 4652, 3528, 4391, 4432, 3924, 4879, 4536, 3741], 3: [42, 74, 29, 48, 45, 26, 41, 40, 29, 26], 4: [132, 212, 352, 312, 249, 133, 198, 115, 179, 131]}

Running analysis for CPT (Holme-Kim)...


...

--- RESULTS ---
{0: [2, 0, 0, 0, 0, 0, 0, 2, 4, 2], 1: [0, 0, 0, 0, 0, 0, 19, 0, 5, 1], 2: [7027, 7127, 7352, 7981, 7817, 8342, 7451, 7468, 7485, 7318], 3: [143, 190, 177, 134, 184, 83, 72, 130, 147, 484], 4: [2044, 1899, 1687, 1101, 1215, 791, 1674, 1616, 1575, 1411]}

Running analysis for MPT (SDA)...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [06:57<00:00, 41.78s/it]


--- RESULTS ---
{0: [2740, 2903, 2590, 2313, 2732, 2315, 2552, 2380, 2937, 2666], 1: [2161, 2143, 2444, 2121, 2439, 2010, 2396, 2291, 2189, 2435], 2: [4061, 3801, 3684, 4133, 3639, 4378, 3900, 3949, 3856, 3824], 3: [114, 117, 158, 188, 106, 174, 88, 124, 87, 116], 4: [140, 252, 340, 461, 300, 339, 280, 472, 147, 175]}

Running analysis for CPT (SDA)...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [04:02<00:00, 24.30s/it]

--- RESULTS ---
{0: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 2: [5588, 5800, 5943, 5423, 4753, 5165, 5337, 5432, 5113, 4753], 3: [1088, 894, 1233, 964, 850, 1213, 950, 1035, 1094, 1096], 4: [1516, 1498, 1016, 1805, 2589, 1814, 1905, 1725, 1985, 2343]}



In [64]:
with open("category_count_results_individual.pickle", "wb") as f:
    pickle.dump(RESULTS, f)

# Analysis at community level

In [65]:
# problem definition
PROBLEM = {
    "num_vars" : 6,
    "names"    : ["project_cost",
                  "gain_right",
                  "alpha_beta",
                  "prob_left",
                  "init_w_scale",
                  "poisson_scale"],
    "bounds"   : [[0.01, 2.00],
                  [1.70, 2.30],
                  [0.70, 0.80],
                  [0.30, 0.45],
                  [0.01, 0.15],
                  [8.00, 20.0]]
}
# generate Saltelli samples
NUM_SAMPLES = 1024
X_8192 = saltelli.sample(PROBLEM, NUM_SAMPLES, calc_second_order=False)


# problem definition
PROBLEM = {
    "num_vars" : 7,
    "names"    : ["project_cost",
                  "gain_right",
                  "alpha_beta",
                  "prob_left",
                  "init_w_scale",
                  "risk_scale",
                  "poisson_scale"],
    "bounds"   : [[0.01, 2.00],
                  [1.70, 2.30],
                  [0.70, 0.80],
                  [0.30, 0.45],
                  [0.01, 0.15],
                  [5.00, 20.0],
                  [8.00, 20.0]]
}
# generate Saltelli samples
NUM_SAMPLES = 1024
X_9216 = saltelli.sample(PROBLEM, NUM_SAMPLES, calc_second_order=False)

/var/folders/bj/4q98hxqx4vq6wwm8qqllvxmr0000gn/T/ipykernel_32258/2087033156.py:19: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  X_8192 = saltelli.sample(PROBLEM, NUM_SAMPLES, calc_second_order=False)
/var/folders/bj/4q98hxqx4vq6wwm8qqllvxmr0000gn/T/ipykernel_32258/2087033156.py:42: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  X_9216 = saltelli.sample(PROBLEM, NUM_SAMPLES, calc_second_order=False)


In [66]:
RESULTS = {}

for dir_idx, D in enumerate(directories):
        
    print(f"Running analysis for {labels[dir_idx]}...")
    
    if "sda" not in D:
        with open(communities_file[dir_idx], "rb") as f:
            communities = pickle.load(f) 
        community_keys = sorted(list(communities.keys()))[:-1]
    
        category_counts = {i:[] for i in range(5)}
        for f in tqdm(os.listdir(D)):
            data = pickle.load(lzma.open(os.path.join(D,f)))
            data_communities = np.zeros((data.shape[0], len(community_keys), data.shape[2])) # 8192, 267, 51
            for c in community_keys:
                for i in range(len(data)):
                    data_communities[i][c] = np.mean(data[i][communities[c]], axis=0)
        
            data = data_communities
            cat1 = category_1(data)
            cat2 = category_2(data, cat1)
            cat3 = category_3(data, cat1+cat2)
            cat4 = category_4(data, cat1+cat2+cat3)
            cat5 = category_5(data, cat1+cat2+cat3+cat4)
            assert len(cat1+cat2+cat3+cat4+cat5) == len(data)

            for i,cat in enumerate([cat1, cat2, cat3, cat4, cat5]):
                category_counts[i].append(len(cat))
                
    else:
        category_counts = {i:[] for i in range(5)}
        for f in tqdm(os.listdir(D)):
            data = pickle.load(lzma.open(os.path.join(D,f))) # (8192, 1250, 51)
            data_communities = []
            
            for i,row in enumerate(data): # for each parameter combination
                
                # get augmented_communities and community keys
                if "cpt" in D:
                    w_scale_param = X_8192[i][4]
                else:
                    w_scale_param = X_9216[i][4]
                    
                with open(f"./sda_graphs/{w_scale_param}.pickle", "rb") as f:
                    _, communities, community_membership, _ = pickle.load(f)
                    
                augmented_communities = {}
                for agent in community_membership:
                    comms = community_membership[agent]
                    for c in comms:
                        if c not in augmented_communities:
                            augmented_communities[c] = {agent}
                        else:
                            augmented_communities[c].add(agent)
                augmented_communities = {k:np.array(list(v)) for k,v in augmented_communities.items()}
                
                community_keys = sorted(list(augmented_communities.keys()))[:-1]
                community_trajectories = []
                for c in community_keys:
                    community_trajectories.append(np.mean(data[i][augmented_communities[c]], axis=0))   
                data_communities.append(np.stack(community_trajectories))
                    
            data = data_communities
            cat1 = category_1(data)
            cat2 = category_2(data, cat1)
            cat3 = category_3(data, cat1+cat2)
            cat4 = category_4(data, cat1+cat2+cat3)
            cat5 = category_5(data, cat1+cat2+cat3+cat4)
            assert len(cat1+cat2+cat3+cat4+cat5) == len(data)

            for i,cat in enumerate([cat1, cat2, cat3, cat4, cat5]):
                category_counts[i].append(len(cat))

    RESULTS[labels[dir_idx]] = category_counts
    print("--- RESULTS ---")
    print(category_counts)
    print()

Running analysis for MPT (Holme-Kim)...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [08:25<00:00, 50.50s/it]


--- RESULTS ---
{0: [7973, 7718, 7770, 8348, 7772, 8200, 8024, 7981, 7676, 7920], 1: [742, 853, 1033, 503, 1024, 550, 905, 949, 797, 863], 2: [348, 442, 193, 260, 248, 324, 152, 55, 379, 278], 3: [78, 87, 92, 84, 81, 62, 56, 93, 115, 86], 4: [75, 116, 128, 21, 91, 80, 79, 138, 249, 69]}

Running analysis for MPT (Random)...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [09:27<00:00, 56.74s/it]


--- RESULTS ---
{0: [7849, 7857, 7310, 7911, 7398, 8229, 7595, 8219, 7677, 8112], 1: [1262, 1260, 1641, 1149, 1558, 899, 1473, 936, 1436, 955], 2: [20, 0, 12, 0, 93, 9, 13, 3, 16, 35], 3: [40, 34, 83, 12, 111, 22, 30, 41, 16, 42], 4: [45, 65, 170, 144, 56, 57, 105, 17, 71, 72]}

Running analysis for CPT (Holme-Kim)...


...

--- RESULTS ---
{0: [1417, 1508, 1350, 1347, 1719, 1497, 1711, 1656, 2111, 1751], 1: [423, 875, 678, 619, 553, 235, 1628, 400, 1600, 596], 2: [5260, 5096, 5381, 6064, 5645, 6648, 4164, 5489, 3908, 5400], 3: [450, 690, 550, 410, 350, 244, 310, 618, 603, 397], 4: [1666, 1047, 1257, 776, 949, 592, 1403, 1053, 994, 1072]}

Running analysis for MPT (SDA)...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [20:08<00:00, 120.84s/it]


--- RESULTS ---
{0: [8034, 7925, 7814, 7463, 7769, 7448, 7836, 7681, 8088, 7891], 1: [1100, 1167, 1295, 1559, 1317, 1621, 1280, 1355, 1019, 1257], 2: [43, 47, 21, 37, 38, 53, 33, 16, 64, 24], 3: [32, 32, 46, 68, 56, 65, 37, 47, 36, 37], 4: [7, 45, 40, 89, 36, 29, 30, 117, 9, 7]}

Running analysis for CPT (SDA)...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [17:15<00:00, 103.51s/it]

--- RESULTS ---
{0: [50, 38, 49, 41, 72, 56, 39, 62, 45, 56], 1: [637, 423, 587, 462, 484, 483, 525, 598, 762, 335], 2: [5625, 5966, 6047, 5584, 4817, 5446, 5416, 5456, 5053, 5156], 3: [1299, 1135, 1159, 1175, 1272, 1394, 1356, 1262, 1360, 1387], 4: [581, 630, 350, 930, 1547, 813, 856, 814, 972, 1258]}



In [68]:
with open("category_count_results_communities.pickle", "wb") as f:
    pickle.dump(RESULTS, f)